In [ ]:
from openai import OpenAI
from typing import List, Optional
from groq import Groq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI
from chaves import *


In [ ]:
with open('all_coments.txt', 'r') as f:
    text = f.read()



text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

sentences=text_splitter.split_text(text)

sentences_paragrafo=text.split('\n')

print(len(sentences))
print(len(sentences_paragrafo))

In [ ]:
client_OpenAI = OpenAI(api_key=OPENAI_API_KEY)

def get_embeddings(texts: List[str], model="text-embedding-3-small", **kwargs) -> List[List[float]]:
    # Replace newlines in each text, which can negatively affect performance.
    texts = [text.replace("\n", " ") for text in texts]

    response = client_OpenAI.embeddings.create(input=texts, model=model, **kwargs)

    return [data.embedding for data in response.data]

# Assuming 'sentences' is a list of sentences
batch_size = 100  # You can adjust this batch size according to your needs
embeddings = []

for i in range(0, len(sentences), batch_size):
    batch = sentences[i:i + batch_size]
    batch_embeddings = get_embeddings(batch)
    embeddings.extend(batch_embeddings)
    print(f"Processed batch {i // batch_size + 1}")

print(embeddings)


In [ ]:
len(embeddings[0])

In [ ]:
import pandas as pd


df_split_langchain = pd.DataFrame()
df_split_langchain['texto'] = sentences
df_split_langchain['values'] = embeddings

def transformar_em_dict(valor):
    return {"texto": valor}

# Aplicar a função à série e atribuir à coluna 'metadata' do df_pinecone
df_split_langchain['texto']=df_split_langchain.texto.replace('\n',' ')
df_split_langchain['metadata'] = df_split_langchain['texto'].apply(transformar_em_dict)

df_split_langchain.drop('texto', axis=1, inplace=True)

df_split_langchain.reset_index( inplace=True)
df_split_langchain.rename(columns={'index': 'id'}, inplace=True)

df_split_langchain.head()
df_split_langchain['id']=df_split_langchain.id.astype(str)

df_split_langchain.to_parquet('df_split_langchain.parquet')

In [ ]:
import pandas as pd


df_split_langchain=pd.read_parquet('df_split_langchain.parquet')

In [ ]:
df_split_langchain

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=API_KEY_PINECONE)

index_name = 


TEXT_EMBEDDING_SIZE=1536
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=TEXT_EMBEDDING_SIZE,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


index = pc.Index(index_name)

index.upsert_from_dataframe(df=df_split_langchain,batch_size=100)

In [ ]:

client_OpenAI = OpenAI(api_key=OPENAI_API_KEY)
def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client_OpenAI.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

def similares(text):
    try:
        embeddings=get_embedding(text)
        index = pc.Index(index_name)
        query=index.query(
            vector=embeddings,
            top_k=3,
            include_metadata=True
        )

        textos_similares=[]
        for matches in query['matches']:
            textos_similares.extend(matches['metadata']['texto'])

        return textos_similares
    except Exception as e:
        print(e)



client_Groq = Groq(
    api_key=GROQ_API_KEY,
)

def chat_llama_rag(text):

    textos_similares=similares(text)
    
    chat_completion = client_Groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                'content':textos_similares[0],
                'content':textos_similares[1],
                'content':textos_similares[2],
                'content':'Dado esses contextos acima responda a pergunta?',
            },
            {
                'role':'user',
                'content':text
            }
        ],
        model="llama3-8b-8192",
    )
    print(chat_completion.choices[0].message.content)


In [ ]:
client_OpenAI = OpenAI(api_key=OPENAI_API_KEY)
def chat_gpt_rag(text):
    # Função fictícia que retorna textos semelhantes
    textos_similares = similares(text)
    
    # Preparando as mensagens para o chat
    messages = [
        {"role": "system", "content": textos_similares[0]},
        {"role": "system", "content": textos_similares[1]},
        {"role": "system", "content": textos_similares[2]},
        {"role": "system", "content": "Dado esses contextos acima responda a pergunta?"},
        {"role": "user", "content": text}
    ]
    
    completion = client_OpenAI.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)   
    print(completion.choices[0].message)
   

In [ ]:
texto='Eu quero comprar um carro 1.0 qual comprar?'
#chat_llama_rag(texto)
chat_gpt_rag(texto)